In [6]:
import numpy as np
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

In [13]:
df = pd.read_csv("balance.txt", sep='\s+')

In [18]:
df.isnull().sum()

Balance      0
Income       0
Limit        0
Rating       0
Cards        0
Age          0
Education    0
Gender       0
Student      0
Married      0
Ethnicity    0
dtype: int64

In [19]:
df.drop(['Limit', 'Age'], inplace=True, axis=1)

In [20]:
df.replace('African American', 'African')

,Balance,Income,Rating,Cards,Education,Gender,Student,Married,Ethnicity
0,12.240798,14.891,283,2,11,Male,No,Yes,Caucasian
1,23.283334,106.025,483,3,15,Female,Yes,Yes,Asian
2,22.530409,104.593,514,4,11,Male,No,No,Asian
3,27.652811,148.924,681,3,11,Female,No,No,Asian
4,16.893978,55.882,357,2,16,Male,No,Yes,Caucasian
...,...,...,...,...,...,...,...,...,...
395,8.764984,12.096,307,3,13,Male,No,Yes,Caucasian
396,9.943838,13.364,296,5,17,Male,No,No,African
397,14.882078,57.872,321,5,12,Female,No,Yes,Caucasian
398,12.001071,37.728,192,1,13,Male,No,Yes,Caucasian


In [22]:
grouped = df.groupby('Ethnicity')
grouped.get_group('Asian')

,Balance,Income,Rating,Cards,Education,Gender,Student,Married,Ethnicity
1,23.283334,106.025,483,3,15,Female,Yes,Yes,Asian
2,22.530409,104.593,514,4,11,Male,No,No,Asian
3,27.652811,148.924,681,3,11,Female,No,No,Asian
7,14.576204,71.408,512,2,9,Male,No,No,Asian
12,19.218800,80.616,394,1,7,Female,No,Yes,Asian
...,...,...,...,...,...,...,...,...,...
385,11.638653,26.400,398,3,15,Female,No,No,Asian
387,9.460031,16.529,126,3,9,Male,No,No,Asian
390,27.079396,135.118,747,3,15,Female,No,Yes,Asian
392,10.872064,25.974,196,2,10,Male,No,No,Asian


In [77]:
df2 = pd.read_csv('store_income.csv')
df2.head()


,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,14 July 2006,UK
1,2,Nordson Corporation,NaN,Tools,$41744177.01,3 December 2006,united states of america
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12 August 2003,UNITED STATES
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,26 October 2006,UK
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,24 December 1973,UK


In [78]:
df2.isnull().sum()

id                 0
store_name         0
store_email      587
department        27
income             0
date_measured      0
country            0
dtype: int64

In [36]:

department_groups = df2.groupby('department')
department_groups['store_email'].count()
department_groups.count()

,id,store_name,store_email,income,date_measured,country
department,,,,,,
Automotive,44,44,18,44,44,44
Baby,52,52,22,52,52,52
Beauty,40,40,12,40,40,40
Books,45,45,26,45,45,45
Clothing,44,44,15,44,44,44
Computers,47,47,17,47,47,47
Electronics,50,50,21,50,50,50
Games,56,56,22,56,56,56
Garden,50,50,19,50,50,50


In [79]:
# set seed for reproducability 
np.random.seed(0)

In [81]:
df2['country'] = df2['country'].str.lower().str.strip()

In [82]:
def replace_to_match(df, columns, string_to_match, min_ratio=90): 
    strings = df[columns].unique()
    matches = fuzzywuzzy.process.extract(string_to_match, strings, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
        # Only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

        # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[columns].isin(close_matches)

        # Replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, columns] = string_to_match

In [83]:
replace_to_match(df2, 'country', "uk")
replace_to_match(df2, 'country', "united kingdom")
replace_to_match(df2, 'country', "united states")
replace_to_match(df2, 'country', "united states of america")
replace_to_match(df2, 'country', "south africa")

In [84]:
df2['country'].unique()

array(['uk', 'united states of america', 'united states', 'south africa',
       'united kingdom'], dtype=object)

In [85]:
df2['country'].replace('united kingdom', 'uk', inplace=True)
df2['country'].replace('united states of america', 'united states', inplace=True)

/var/folders/j2/d48756hs4sngfql_m_100d8w0000gn/T/ipykernel_6882/2696712290.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['country'].replace('united kingdom', 'uk', inplace=True)
/var/folders/j2/d48756hs4sngfql_m_100d8w0000gn/T/ipykernel_6882/2696712290.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting v

Working with Date and Time: 
Challenges
1. Weekdays and timezones may differ among regions
2. Leap years

In [60]:
from datetime import date

Pandas has a dtype called datetime64 described in the documntation, but if you were to call the dtype of a specific column by indexing it before .dtypes, you would receive code that matches a dtype described in the numpy documentation.

Datetime dtype conversion is called parsing because it takes a string and identifies the components that match the strfdirective: in other words, the string is evaluated and outputs a datetime64 element according to the date/time format you want, based on punctuation and forms of dates/times.  

Using the pd.to_datetime( .... )
The right data format can be specified using the parameter format= 
Alternatively, using the parameter infer_datetime_format=True
- Slower than directly specifying formats 
- Won't always be accurate (depends on data entry)

In [89]:
df2.dtypes

id                        int64
store_name               object
store_email              object
department               object
income                   object
date_measured            object
country                  object
date_parsed      datetime64[ns]
dtype: object

In [87]:
df2['date_parsed'] = pd.to_datetime(df2['date_measured'], format='mixed')